# Topics Covered

# Managed vs External Tables

In [ ]:
#see configs

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession\
                    .builder\
                    .master("spark://spark-master:7077")\
                    .appName("Day_7")\
                    .config("spark.sql.warehouse.dir", "/data/spark-warehouse")\
                    .getOrCreate()

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [18]:
orders_schema_struct = StructType(
                                    [
                                        StructField("order_id",IntegerType()),
                                        StructField("customer_id",IntegerType()),
                                        StructField("product_id",IntegerType()),
                                        StructField("price",FloatType()),
                                        StructField("order_date",DateType()),
                                        StructField("order_status",StringType()),
                                        StructField("state",StringType()),
                                        StructField("quantity",IntegerType()),
                                    ]
                                )

df_orders = spark.read.csv('/data/orders_50mb.csv',schema=orders_schema_struct,header=True)

df_orders.show(5)

df_orders.createOrReplaceTempView('orders_table')

+--------+-----------+----------+------+----------+------------+-------+--------+
|order_id|customer_id|product_id| price|order_date|order_status|  state|quantity|
+--------+-----------+----------+------+----------+------------+-------+--------+
|       1|     192520|        70|152.33|2007-10-10|      PLACED| Odisha|       2|
|       2|     835421|        34|117.42|2008-12-02|     SHIPPED| Kerala|       3|
|       3|     159165|        13|128.85|2010-04-22|   DELIVERED|Gujarat|       3|
|       4|     403890|        25|195.71|2007-05-30|   CANCELLED|  Bihar|       1|
|       5|     273746|        41|125.08|2003-12-13|     SHIPPED| Odisha|       3|
+--------+-----------+----------+------+----------+------------+-------+--------+
only showing top 5 rows



# create database

In [11]:
spark.sql(
    '''show databases'''
).show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [12]:
spark.sql(
    '''
    create database gyanDb
    '''
)
# here gyanDb will be store in spark-warehouse and its entry/metadata in metastore

26/02/01 17:37:49 WARN ObjectStore: Failed to get database gyandb, returning NoSuchObjectException
26/02/01 17:37:49 WARN ObjectStore: Failed to get database gyandb, returning NoSuchObjectException
26/02/01 17:37:49 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
26/02/01 17:37:49 WARN ObjectStore: Failed to get database gyandb, returning NoSuchObjectException


DataFrame[]

In [13]:
spark.sql(
    '''show databases'''
).show()

+---------+
|namespace|
+---------+
|  default|
|   gyandb|
+---------+



# create table

In [15]:
spark.sql(
    '''
    create table gyanDb.emp(empid int,salary int)
    '''
)

26/02/01 17:41:08 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
26/02/01 17:41:08 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
26/02/01 17:41:08 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
26/02/01 17:41:08 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/02/01 17:41:08 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
26/02/01 17:41:08 WARN HiveMetaStore: Location: file:/data/spark-warehouse/gyandb.db/emp specified for non-external table:emp


DataFrame[]

# insert data

In [16]:
spark.sql(
    '''
    insert into gyanDb.emp values(1,10000)
    '''
)
# part file will be created inside gyanDb/emp

DataFrame[]

# create table using format

In [17]:
spark.sql(
    '''
    create table gyanDb.emp_new(id int,salary int) Using parquet
    '''
)

DataFrame[]

In [18]:
spark.sql(
    '''
    insert into gyanDb.emp_new values(1,10000)
    '''
)
# data stored in parquet format

DataFrame[]

In [19]:
spark.sql(
    '''
    insert into gyanDb.emp_new values(2,20000)
    '''
)
#another parquet file

DataFrame[]

# Delete/update/truncate not supported in managed table

its bcz the data are written into disk in parquet/csv format,it cant be deleted/updated

In [20]:
spark.sql(
    '''
    update gyanDb.emp_new set salary=30000 where id=1
    '''
)

UnsupportedOperationException: UPDATE TABLE is not supported temporarily.

# more

In [22]:
spark.sql('''describe gyanDb.emp_new''').show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      id|      int|   NULL|
|  salary|      int|   NULL|
+--------+---------+-------+



In [24]:
spark.sql('''describe extended gyanDb.emp_new''').show(truncate=False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|id                          |int                                                           |NULL   |
|salary                      |int                                                           |NULL   |
|                            |                                                              |       |
|# Detailed Table Information|                                                              |       |
|Catalog                     |spark_catalog                                                 |       |
|Database                    |gyandb                                                        |       |
|Table                       |emp_new                                             

# catalog

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession\
                    .builder\
                    .master("spark://spark-master:7077")\
                    .appName("Day_7")\
                    .config("spark.sql.warehouse.dir", "/data/spark-warehouse")\
                    .config("spark.sql.catalogImplementation", "hive")\
                    .config("javax.jdo.option.ConnectionURL", "jdbc:derby:/data/metastore_db;create=true")\
                    .getOrCreate()


In [5]:
spark.stop()

In [7]:
spark.sql('''show databases''').show()

26/02/01 18:04:25 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/02/01 18:04:25 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
26/02/01 18:04:27 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
26/02/01 18:04:27 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.5
26/02/01 18:04:27 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


+---------+
|namespace|
+---------+
|  default|
+---------+



In [8]:
spark.sql('''create database mydb''')

++
||
++
++



26/02/01 18:04:54 WARN ObjectStore: Failed to get database mydb, returning NoSuchObjectException
26/02/01 18:04:54 WARN ObjectStore: Failed to get database mydb, returning NoSuchObjectException
26/02/01 18:04:54 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
26/02/01 18:04:54 WARN ObjectStore: Failed to get database mydb, returning NoSuchObjectException


In [9]:
spark.sql('''show databases''').show()

+---------+
|namespace|
+---------+
|  default|
|     mydb|
+---------+



In [10]:
spark.sql('''create table mydb.emp(id int,salary int)''')

26/02/01 18:05:37 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
26/02/01 18:05:37 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
26/02/01 18:05:38 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
26/02/01 18:05:38 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/02/01 18:05:38 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
26/02/01 18:05:38 WARN HiveMetaStore: Location: file:/data/spark-warehouse/mydb.db/emp specified for non-external table:emp


DataFrame[]

In [12]:
spark.sql('''drop table mydb.emp''')

DataFrame[]

In [14]:
spark.sql('''create table mydb.emp(id int ,salary int) USING PARQUET''')

DataFrame[]

In [15]:
spark.sql('''insert into mydb.emp values(1,10000)''')

DataFrame[]

# create table from df

In [19]:
orders_schema_struct = StructType(
                                    [
                                        StructField("order_id",IntegerType()),
                                        StructField("customer_id",IntegerType()),
                                        StructField("product_id",IntegerType()),
                                        StructField("price",FloatType()),
                                        StructField("order_date",DateType()),
                                        StructField("order_status",StringType()),
                                        StructField("state",StringType()),
                                        StructField("quantity",IntegerType()),
                                    ]
                                )

df_orders = spark.read.csv('/data/orders_50mb.csv',schema=orders_schema_struct,header=True)


In [20]:
# create a managed table orders in default db.
df_orders.write.saveAsTable("orders") #it will be inside spark-warehouse

In [21]:
spark.sql('''select * from orders''').show(5)

+--------+-----------+----------+------+----------+------------+-------+--------+
|order_id|customer_id|product_id| price|order_date|order_status|  state|quantity|
+--------+-----------+----------+------+----------+------------+-------+--------+
|       1|     192520|        70|152.33|2007-10-10|      PLACED| Odisha|       2|
|       2|     835421|        34|117.42|2008-12-02|     SHIPPED| Kerala|       3|
|       3|     159165|        13|128.85|2010-04-22|   DELIVERED|Gujarat|       3|
|       4|     403890|        25|195.71|2007-05-30|   CANCELLED|  Bihar|       1|
|       5|     273746|        41|125.08|2003-12-13|     SHIPPED| Odisha|       3|
+--------+-----------+----------+------+----------+------------+-------+--------+
only showing top 5 rows



In [22]:
#append mode
# i have 4 parquet file,so after this i will have 8 file
df_orders.write.mode('append').saveAsTable("orders")

In [23]:
#append mode
# i have 8 parquet file,so after this i will have 4 file
df_orders.write.mode('overwrite').saveAsTable("orders")

# External Table

In [25]:
spark.sql('''CREATE TABLE orders_ext (
id INT,
amount DOUBLE
)
USING PARQUET
LOCATION '/data/orders_external'
''')

26/02/01 18:26:20 WARN HadoopFSUtils: The directory file:/data/orders_external was not found. Was it deleted very recently?


DataFrame[]

In [26]:
spark.sql('''insert into orders_ext values(1,10000)''')

DataFrame[]

In [28]:
spark.sql('''describe extended orders_ext''').show(truncate=False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|id                          |int                                                           |NULL   |
|amount                      |double                                                        |NULL   |
|                            |                                                              |       |
|# Detailed Table Information|                                                              |       |
|Catalog                     |spark_catalog                                                 |       |
|Database                    |default                                                       |       |
|Table                       |orders_ext                                          

26/02/01 19:06:05 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-6ce02b27-bee1-4561-9d65-00e4a0e7fbde/pyspark-41dbdd97-d9d5-4459-985d-e35db630338f. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-6ce02b27-bee1-4561-9d65-00e4a0e7fbde/pyspark-41dbdd97-d9d5-4459-985d-e35db630338f
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun

In [ ]:
spark.sql('''drop table orders_ext''') # it will not delete the data